In [5]:
import pandas as pd
df = pd.read_csv('../data/tripmap_ex.csv')
df.head()

,X,start_lng,start_lat,end_lng,end_lat
0,A,-122.422449,37.769729,-122.429596,37.767726
1,B,-122.429596,37.767726,-122.436909,37.757346
2,C,-122.436909,37.757346,-122.432505,37.757018
3,D,-122.432505,37.757018,-122.423749,37.763298
4,E,-122.423749,37.763298,-122.428622,37.763461


In [8]:
import h3

df['hex_id'] = 0

for ind in df.index:
    df['hex_id'][ind] = h3.geo_to_h3( df['start_lat'][ind], df['start_lat'][ind], 8)
df.head()

/var/folders/5t/28jzvs016nq5vxslb670gs040000gn/T/ipykernel_4184/3813553967.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hex_id'][ind] = h3.geo_to_h3( df['start_lat'][ind], df['start_lat'][ind], 8)
/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,X,start_lng,start_lat,end_lng,end_lat,hex_id
0,A,-122.422449,37.769729,-122.429596,37.767726,882dad71bdfffff
1,B,-122.429596,37.767726,-122.436909,37.757346,882dad71bdfffff
2,C,-122.436909,37.757346,-122.432505,37.757018,882dad71a1fffff
3,D,-122.432505,37.757018,-122.423749,37.763298,882dad71a1fffff
4,E,-122.423749,37.763298,-122.428622,37.763461,882dad71abfffff


In [9]:
df.to_csv('tripmap_ex_h3.csv')